In [ ]:
!nvidia-smi

In [ ]:
import cv2
import torch
import numpy as np
from PIL import Image
from sam3.model_builder import build_sam3_image_model
from sam3.model.sam3_image_processor import Sam3Processor
from utils import *
from vis_utils import *
import os
import json

In [ ]:
from huggingface_hub import login
hf_token = ''
with open('hf_token.txt', 'r') as token_file:
    hf_token = token_file.read()
login(hf_token)

In [ ]:
#getting dataset info
data_path = "datasets/"
dataset_name = "BIG"
dataset_info_path = "%s/%s.json" %(data_path, dataset_name)
with open (dataset_info_path,'r') as file:
    dataset_info = json.load(file)

In [ ]:
#initialising model
model = build_sam3_image_model()
processor = Sam3Processor(model,resolution=1008)

In [ ]:
def inference_on_image_with_text(image_processor,img,text_prompt):
    '''
    Run sam3 inference on image with text prompt
    Returns:
        sem_mask - cv2 image(bool), instance mask - cv2 image (uint8), overlayed image - cv2 image uint8
    '''
    #convert image to PIL
    image = cv2pil(img)
    inference_state = processor.set_image(image)
    # Prompt the model with text
    output = processor.set_text_prompt(state=inference_state, prompt=text_prompt)
    # Get the masks, bounding boxes, and scores
    masks, boxes, scores = output["masks"], output["boxes"], output["scores"]
    sem_mask, instance_mask, overlayed = visualize(img,masks[...,0,:,:].cpu().detach().numpy())
    del masks
    del boxes
    del scores
    return sem_mask, instance_mask, overlayed

In [ ]:
ious = []
for data in dataset_info:
    image = cv2.imread(data['img'])
    gt_mask = cv2.imread(data['mask'])
    prompt = data['prompt']
    sem_mask, instance_mask, overlayed = inference_on_image_with_text(processor,image,prompt)
    iou = calc_iou(sem_mask,gt_mask[:,:,0])
    ious.append(iou)
    

In [ ]:
mean_iou = np.mean(ious)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(gt_mask)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(cv2pil(overlayed))